In [2]:
import os
os.environ['KAGGLE_USERNAME']="simransohal"
os.environ['KAGGLE_KEY']="3aa7697cd9a4ba2a3715c4a29c0f0cb6"

In [3]:
!kaggle datasets download -d mrgeislinger/asl-rgb-depth-fingerspelling-spelling-it-out

 99% 2.09G/2.11G [00:23<00:00, 122MB/s]
100% 2.11G/2.11G [00:23<00:00, 97.8MB/s]


In [4]:
ls

asl_alphabet_test/   asl-rgb-depth-fingerspelling-spelling-it-out.zip
asl_alphabet_train/  sample_data/
asl-alphabet.zip


In [5]:
import zipfile
with zipfile.ZipFile('asl-rgb-depth-fingerspelling-spelling-it-out.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [6]:
ls

asl_alphabet_test/   asl-rgb-depth-fingerspelling-spelling-it-out.zip
asl_alphabet_train/  dataset5/
asl-alphabet.zip     sample_data/


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.optim import Adam

In [8]:
d=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
model=models.resnet18(pretrained=False)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
ls

asl_alphabet_test/   asl-rgb-depth-fingerspelling-spelling-it-out.zip
asl_alphabet_train/  dataset5/
asl-alphabet.zip     sample_data/


In [11]:
#Getting the number of input features in fully connected layer
#Model is to be classified in just three layers
#Created a new layer and replaced the fully connected layer with it
num_classes=24
features=model.fc.in_features
model.fc=nn.Linear(features,num_classes)

In [12]:
model=model.to(d)

In [13]:
train_transformations=transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])
test_transformation=transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])])

In [15]:
dataset = datasets.ImageFolder(r"dataset5", transform=train_transformations)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [13]:
#data=datasets.ImageFolder(r"asl_alphabet_train",transform=train_transformations)

In [14]:
#test_data=datasets.ImageFolder(r"asl_alphabet_test",transform=test_transformation)

In [16]:
optimizer=Adam(model.parameters(),lr=0.001)
criterion=nn.CrossEntropyLoss()

In [17]:
from torch.utils.data import DataLoader
loader_train=DataLoader(train_dataset,batch_size=128,shuffle=True )
loader_test=DataLoader(test_dataset,batch_size=128,shuffle=True)

In [ ]:
num_epochs = 10
total_steps = len(loader_train)

for epoch in range(num_epochs):
    for i, data in enumerate(loader_train):
        images, labels = data[0].to(d), data[1].to(d)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Backprop and optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Train accuracy
        total = labels.size(0)
        _,predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        if (i + 1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                .format(epoch + 1, num_epochs, i + 1, total_steps, loss.item(),
                    (correct / total) * 100))
            


In [ ]:
model.eval()
with torch.no_grad():
  correct = 0
  total=0
  for image,label in loader_test:
    images,label=images.to(d),labels.to(d)
    result=model(images)
    _,predicted=torch.max(result.data,1)
    total=total+labels.size(0)
    correct=correct+(predicted==labels).sum().items()
    print(f'Accuracy {100 * correct / total:.2f}%')


